In [ ]:
##### Working Code ####
import notebookutils, csv
import os, json, time, pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from openai import AzureOpenAI


areas = ["Employment"]
# , "Information and Communication Technology"]
#        "Communications other than ICT",
#         "Design and Delivery of Programs and Services", "Procurement of goods, services and facilities",
#        "Transportation", "Built Environment"]

os.environ["AZURE_OPENAI_API_KEY"] = "xxxxxxxxxx"
os.environ["AZURE_OPENAI_ENDPOINT"] = "xxxxxxxxxxxx"
os.environ["OPENAI_API_VERSION"] = "2024-02-01"


StatementMeta(, 7cf4afe6-0fa5-45a6-91c8-db5db7343901, 7, Finished, Available, Finished)

In [ ]:
# Working Code

def json_extract(obj, key):
    """Recursively fetch values from nested JSON."""
    arr = []

    def extract(obj, arr, key):
        """Recursively search for values of key in JSON tree."""
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr, key)
                elif k == key:
                    arr.append(v)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    values = extract(obj, arr, key)
    return values[0]

def extract_metrics(sector, area_desc_pr, area_desc_pl, org_c, area_m, processed_file_path):
    
    res = []
    for i in range(len(org_c)):

        prompt = f"Please compare the plan and the progress report for {area_m} of {org_c[i]} in {sector} as per the details in the plan : {area_desc_pl[i]} and the progress report: {area_desc_pr[i]}." 
        prompt = prompt + f"Calculate an overall progress made in this area for accessibility improvements"
        prompt = prompt + "Provide the output in the format : "
        prompt = prompt + "{ """"Organization"" : """", ""Sector"" : """", ""Area"" : """", ""Analysis"" : """", ""Explanation"" : """", ""Progress"" : [ { ""Activity"" : """", ""Percent Progress"" : """"}], ""Total percent progress"" : """", ""Explanation of Percent Progress"" : """" }"
    
        MESSAGES = [
            {"role": "system", "content": "You are an accessibility compliance analyst."},
            {"role": "user", "content": prompt},
        ]

        time.sleep(60)
        
        client = AzureOpenAI()

        # temp is set to 0 to get s consistent response from the model
        response = client.chat.completions.create(
            model="gpt-4-turbo-2024-04-09",
            temperature=0.0,
            messages=MESSAGES
        )
        parsed_data = json.loads(response.json())
        content = json_extract(parsed_data, "content")
        
        time.sleep(60)
        create_or_append_json(content , processed_file_path)     
    return(content)
    


def create_or_append_json(content, processed_file_path):

    new_data = content

    # Check if file exists and has content
    if notebookutils.fs.exists("file:" + processed_file_path):
        # Read existing data
        with open(processed_file_path, 'r') as file:
            try:
                data = json.load(file)
                
                # Handle different data structures
                if isinstance(data, list):
                    data.append(new_data)
                elif isinstance(data, dict):
                    if 'items' not in data:
                        data['items'] = []
                    data['items'].append(new_data)
                else:
                    # If neither list nor dict, wrap existing data and new data in a list
                    data = [data, new_data]
                    
            except json.JSONDecodeError:
                print("Error: File contains invalid JSON. Creating new file...")
                data = {'items': [new_data]}
    else:
        # Create new file with initial data
        data = {'items': [new_data]}
    
    # Write data back to file
    with open(processed_file_path, 'w') as file:
        json.dump(data, file, indent=4)


StatementMeta(, 7cf4afe6-0fa5-45a6-91c8-db5db7343901, 8, Finished, Available, Finished)

In [ ]:
metrics_file_path = "/lakehouse/default/Files/Gold/metrics.json"

area_m = 'employment'
area_desc_pr = []
area_desc_pl = []
org_c = []


df_org = spark.sql("SELECT DISTINCT LOWER(org) as org FROM Accessibility_LH.dbo.Plan_Area_Extract") # WHERE lower(sector) = 'telecom' and lower(org)='<orgname>'")

#display(df_org)
# convert column to list
orgs = df_org.select("org").rdd.flatMap(lambda x: x).collect()
#print(orgs)

for org_filter in orgs:
    for area_m in areas:
        sql_plan = f"SELECT * FROM Accessibility_LH.dbo.Plan_Area_Extract WHERE LOWER(org) = '{org_filter}' and LOWER(area) = LOWER('{area_m}')"
        sql_prog_rep = f"SELECT * FROM Accessibility_LH.dbo.Progressreport_Area_Extract WHERE LOWER(org) = '{org_filter}' and LOWER(area) = LOWER('{area_m}')"

        df_plan = spark.sql(sql_plan)
        df_prog_rep = spark.sql(sql_prog_rep)

        #display(df_prog_rep)
        #display(df_plan)

        org_c.append(org_filter)
        sector = df_plan.first()['sector']
        area_desc_pr.append(df_prog_rep.first()['area_desc'])
        area_desc_pl.append(df_plan.first()['area_desc'])
        response = extract_metrics(sector, area_desc_pr, area_desc_pl, org_c, area_m, metrics_file_path)
        print(response)
#print(f"{area_desc_pr} \n\n")
#print (f"{area_desc_pl} \n\n")


StatementMeta(, 7cf4afe6-0fa5-45a6-91c8-db5db7343901, 12, Finished, Available, Finished)

```json
{
  "Organization": "Rogers in Telecom",
  "Sector": "Telecommunications",
  "Area": "Employment",
  "Analysis": "Comparison of Planned Tasks and Progress",
  "Explanation": "The progress report for 2023 outlines several initiatives that align with the planned tasks for 2024 and 2025, focusing on enhancing accessibility for employees with accessibility needs. The initiatives include recruitment process enhancements, development of an accessibility hub, efficiencies in medical accommodations, and a pilot project for ergonomic equipment.",
  "Progress": [
    {
      "Activity": "Develop a plan to improve the accommodation process for employees",
      "Percent Progress": 50
    },
    {
      "Activity": "Create an assistive technology service catalogue",
      "Percent Progress": 0
    },
    {
      "Activity": "Create a support model for employees with accessibility needs",
      "Percent Progress": 0
    },
    {
      "Activity": "Include accessibility skills in job ads whe